In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import os
import sys
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)
from gnn_model.train_model import train_model


In [2]:
import numpy as np
import torch

def projectile_trajectory_2d(m, dt, theta, v0, g = 9.81, x0 = 0, y0 = 0, N=1, dim=2):
    """
    Calculates a 2D cartesian trajectory of a projectile of mass m launched at angle theta with initial velocity v0.  
    
        Parameters:
    theta (int or float): angle between projectile trajectory and x-axis at t=0s in degrees \\
    v0 (int or float): the intial velocity in m/s \\
    m (int or float): mass of the projectile in kg \\
    dt (int or float): time step for simulation in s \\
    g (int or float): gravity constant (standard is 9.81) in m/s^2 \\
    x0, y0 (int or float): initial position of ball in m \\

        Returns: 
    The output is a dictionary containing the {time, positions, velocities, masses} coordinates of the projectile
    """

    # Convert theta to radians
    theta_rad = np.radians(theta)

    # Initial velocity components
    vx0 = v0 * np.cos(theta_rad)
    vy0 = v0 * np.sin(theta_rad)

    # Time array
    t_max = (vy0 + np.sqrt(vy0**2+2*g*y0)) / g
    t = np.arange(0, t_max, dt)
    T = len(t)
    print(T)
    
    # Compute trajectory
    x = x0 + vx0 * t
    y = y0 + vy0 * t - 0.5 * g * t**2

    # Remove negative y values and make np array
    valid = y >= 0
    x, y, t = x[valid], y[valid], t[valid]
    trajectory = np.column_stack((x,y))

    vx = np.full_like(x, vx0)
    vy = vy0 - g * t
    trajectory_velocities = np.column_stack((vx,vy))
    
    trajectory_data = {
        "time": torch.arange(T, dtype=torch.float32),
        "positions": torch.zeros((T, N, dim), dtype=torch.float32),
        "velocities": torch.zeros((T, N, dim), dtype=torch.float32),
        "masses": torch.arange(N, dtype=torch.float32)
    }
    
    trajectory_data["time"] = torch.tensor(t)
    trajectory_data["positions"] = torch.tensor(trajectory[:, :].reshape(T, N, dim))
    trajectory_data["velocities"] = torch.tensor(trajectory_velocities[:, :].reshape(T, N, dim))
    trajectory_data["masses"] = torch.tensor(m)

    return trajectory_data


In [4]:
import torch
from torch_geometric.data import Data

def node_data_list(trajectory_dict, self_loop=True, complete_graph=True):
    """
    Converts a trajectory dictionary into a list of PyTorch Geometric `Data` objects 
    representing graph-structured data for a node-based learning task.

    Parameters:
    -----------
    trajectory_dict : dict
        A dictionary containing simulation data with the following keys:
        - "masses" (torch.Tensor): Mass values for the nodes (particles).
        - "time" (torch.Tensor): Time steps in the trajectory.
        - "positions" (list of torch.Tensor): List of position tensors at each time step.
        - "velocities" (list of torch.Tensor): List of velocity tensors at each time step.

    self_loop : bool, optional (default=True)
        If True, self-loops (edges from a node to itself) are included in the graph.

    complete_graph : bool, optional (default=True)
        If True, a fully connected graph is created where each node is connected to every other node.

    Returns:
    --------
    data_list : list of torch_geometric.data.Data
        A list of `Data` objects, each representing a graph at a given time step.
        Each `Data` object contains:
        - `x` (torch.Tensor): Node features with shape (N, num_features), where `num_features` includes
          position, velocity, mass, and time.
        - `y` (torch.Tensor): Target values representing velocity updates.
        - `edge_index` (torch.Tensor): Graph connectivity in COO format with shape (2, num_edges).

    Notes:
    ------
    - The function assumes a constant number of nodes (N) throughout the trajectory.
    - If `masses` is a scalar, it is broadcasted to all nodes.
    - Edge indices are stored in COO format (two-row tensor).

    Example:
    --------
    >>> trajectory_dict = {
    ...     "masses": torch.tensor(1.0),
    ...     "time": torch.arange(3),
    ...     "positions": [torch.rand(5, 2) for _ in range(3)],
    ...     "velocities": [torch.rand(5, 2) for _ in range(3)]
    ... }
    >>> graphs = node_data_list(trajectory_dict)
    >>> print(graphs[0])
    Data(x=[5, 6], y=[5, 2], edge_index=[2, 25])
    """

    data_list = []

    N = trajectory_dict["masses"].numel()
    # Ensure masses has the correct shape (N, 1)
    mass_data = trajectory_dict["masses"]

    # If masses is scalar (0-dimensional), repeat it for N particles
    if mass_data.dim() == 0:  # scalar case
        mass_data = mass_data.repeat(N, 1)  # shape (N, 1)

    # If masses is already a 1D tensor of shape (N,)
    elif mass_data.dim() == 1:  # 1D tensor case
        mass_data = mass_data.unsqueeze(1)  # shape (N, 1)
    
    for i in range(len(trajectory_dict["time"]) - 1):
        time_feature = trajectory_dict["time"][i].repeat(N,1)
        print(time_feature.shape)
        # position_features = trajectory_dict["positions"][i]

        x_features = torch.cat([
            trajectory_dict["positions"][i],    # (N,dim)
            trajectory_dict["velocities"][i],   # (N, dim)
            mass_data,       # (N, 1)
            time_feature    # (N, 1)
        ], dim=1)   # tensor of shape (N, 2 + 2 + 1 + 1) + (N, 6) if dim = 2

        velocity_update = trajectory_dict["velocities"][i+1] - trajectory_dict["velocities"][i]
        acceleration = velocity_update / (trajectory_dict["time"][i+1] - trajectory_dict["time"][i])

        y_target = torch.tensor(acceleration, dtype=torch.float32)
        # y_target = torch.tensor(velocity_update, dtype=torch.float32)

        edge_list = []
        
        if self_loop:
            edge_list.extend([j,j] for j in range(N))
        
        if complete_graph:
            edge_list.extend([k,j] for k in range(N) for j in range(N) if k != j)           
   
        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()        

        data_list.append(Data(x=x_features.float(), y=y_target, edge_index=edge_index))

    return data_list


In [5]:
traj_proj = projectile_trajectory_2d(3, 0.02, 40, 90)
print(traj_proj["time"].shape)

590
torch.Size([590])


In [6]:
1170/26

45.0

In [7]:
print(traj_proj["masses"].numel())

1


In [8]:
traj_graph = node_data_list(traj_proj, self_loop=True, complete_graph=False)

torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([

/var/folders/_p/ztk2c0b93nbb9q83wg0_pq780000gn/T/ipykernel_59147/2008593403.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)


In [9]:
edge_list = []   
edge_list.extend([i,j] for i in range(3) for j in range(3) if i != j)
# for j in range(3):
#     for k in range(3):
#         if j != k:
#             edge_list.append([j,k])

edge_index = torch.tensor(edge_list, dtype=torch.int64).contiguous()
                

In [10]:
edge_index

tensor([[0, 1],
        [0, 2],
        [1, 0],
        [1, 2],
        [2, 0],
        [2, 1]])

In [11]:
m = np.array([1,2,3])
dict_ex = {
        "masses": torch.arange(3, dtype=torch.float32)
    }
    
dict_ex["masses"] = torch.tensor(m)
mass_data = dict_ex["masses"]

mass_data = mass_data.unsqueeze(1)  # shape (N, 1)
mass_data

tensor([[1],
        [2],
        [3]])

In [12]:
mass_data

tensor([[1],
        [2],
        [3]])

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# define the message passing class to initiate MLP's among the nodes
class GNN_MLP(MessagePassing):
    def __init__(self, n_f, m_dim, hidden_channels, out_channels, single_node = False):
        super(GNN_MLP, self).__init__(aggr='add')  # "Add" aggregation for summing over forces
        
        # initialising the MLP by creating the self.MLP attribute. 2 * in_channels to account for the fact that it may use both it's own and the other nodes features.
        self.mess_mlp = nn.Sequential(
            nn.Linear(2 * n_f, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, m_dim)
        )

        self.agg_mlp = nn.Sequential(
            nn.Linear(m_dim + n_f, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, out_channels)
        )
        self.single_node = single_node

    def forward(self, x, edge_index):
        """
        Forward calls propagate to initiate message passing for all nodes in edge_index
        """
        x=x
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)  # Triggers message passing
    
    def message(self, x_i, x_j):
        """
        Applying mlp to every directed edge in edge_index for [x_i, x_j]
        """
        edge_features = torch.cat([x_i, x_j], dim=1)  # Concatenating node features for edge
        return self.mess_mlp(edge_features)  # Pass through MLP

    def update(self, aggr_out, x=None):
        """
        Updates node features with passed messages.
        """
        if self.single_node:
            return aggr_out
        
        else:
            update_features = torch.cat([x, aggr_out], dim=1)
            return self.agg_mlp(update_features)

In [14]:
single_node =False

if single_node:
    print("True")

In [20]:
model = GNN_MLP(n_f=6, m_dim=2, out_channels=2, hidden_channels=128, single_node=True)

In [18]:
train_model(model, traj_graph, lr=0.01, epochs=100)

GNN_MLP()

In [19]:
# Output results
print("\nSimulation Results:")
for idx, data in enumerate(traj_graph):
    with torch.no_grad():
        pred_force = model(data.x, data.edge_index)  # Model predicts net force
        
        # Ground truth force
        ground_truth_force = data.y
        
        # Ensure correct shapes and conversion to NumPy for printing
        pred_force_np = pred_force.cpu().numpy().flatten()  # Convert to NumPy array and flatten
        ground_truth_np = ground_truth_force.cpu().numpy().flatten()  # Convert to NumPy and flatten

        # Print each component of the force vector
        print(f"Sample {idx + 1}:")
        for i in range(len(pred_force_np)):
            print(f"   Component {i}: Ground Truth = {ground_truth_np[i]:.6f}, Predicted = {pred_force_np[i]:.6f}")



Simulation Results:
Sample 1:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015717
   Component 1: Ground Truth = -9.810000, Predicted = -10.319181
Sample 2:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015718
   Component 1: Ground Truth = -9.810000, Predicted = -10.319779
Sample 3:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015719
   Component 1: Ground Truth = -9.810000, Predicted = -10.320374
Sample 4:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015720
   Component 1: Ground Truth = -9.810000, Predicted = -10.320965
Sample 5:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015720
   Component 1: Ground Truth = -9.810000, Predicted = -10.321551
Sample 6:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015721
   Component 1: Ground Truth = -9.810000, Predicted = -10.322136
Sample 7:
   Component 0: Ground Truth = 0.000000, Predicted = 0.015722
   Component 1: Ground Truth = -9.810000, Predicted = -10.322715
Sample 8:
   Compone

AttributeError: 'Monkey' object has no attribute 'favorite_tree'